# Notebook to train and evaluate Yolo Mlt (Yolov5 small for object detection and image classification)

As of today (December 2nd 2022), the goal is to detect [speed signs, road work signs/objects, the horizon of the road] and to classify the road condition [dry/snowy/wet].

### Imports

In [ ]:
import glob
import os

In [ ]:
pwd

----------------

## Training

### Train the detection task on esmart_wip dataset

Let's start by training the detection task with all layers activated (besides the classification head just as a security).

In [ ]:
!python multitasks/train.py --epochs 30 --img 512 --weights yolo5s.pt --data ../datasets/esmart_wip/data.yaml --batch-size 32 --only_det --freeze 25

In [ ]:
# get the last folder of the folder runs/train-mlt/
list_of_files = glob.glob('runs/train-mlt/*')
latest_exp_path = max(list_of_files, key=os.path.getmtime)
print(latest_exp_path)
last_weights = f"{latest_exp_path}/weights/last.pt"
best_weights = f"{latest_exp_path}/weights/best.pt"

### Train the classification task on esmart_context

Train only the classification using the weights form the previous run and freezing every layer besides the classification head (and optionally the layer before #8)

In [ ]:
!python multitasks/train.py --epochs 30 --img 512 --weights {best_weights} --data ../datasets/esmart_context/data.yaml --batch-size 32 --only_cls --freeze_all_but 8 25 --cut_img 0.5
#!python multitasks/train.py --epochs 50 --img 512 --weights {best_weights} --data ../datasets/esmart_context/data.yaml  --batch-size 32 --only_cls --freeze_all_but 25 --cut_img 0.5

In [ ]:
# get the last folder of the folder runs/train-mlt/
list_of_files = glob.glob('runs/train-mlt/*')
latest_exp_path = max(list_of_files, key=os.path.getmtime)
print(latest_exp_path)
last_weights = f"{latest_exp_path}/weights/last.pt"
best_weights = f"{latest_exp_path}/weights/best.pt"

### Train again the detection task on esmart_wip

Train only the detection using the weights form the previous run and freezing every layer in the backbone and classification head.


!python multitasks/train.py --epochs 15 --img 512 --weights yolov5s.pt --data ../datasets/esmart_wip/data.yaml --batch-size 32 --only_det

In [ ]:
list_of_files = glob.glob('runs/train-mlt/*')
latest_exp_path = max(list_of_files, key=os.path.getmtime)
print(latest_exp_path)
last_weights = f"{latest_exp_path}/weights/last.pt"
best_weights = f"{latest_exp_path}/weights/best.pt"

### OR Train again the detection and classification at the same time on esmart_hybrid

In [ ]:
!python multitasks/train.py --epochs 20 --img 512 --weights yolov5s.pt --data ../datasets/hybrid/data.yaml --batch-size 32

------------------

# Evaluate

In [ ]:
weights_path = 'runs/train-mlt/exp162/weights/last.pt'

### The road conditions classification on esmart_context

Note that the display of the logger works well in the terminal but sometimes doesn't show the title of the metrics when running from a notebook.
Here is the list of the reported metrics (in order):

*Class, Images, Instances, cls_P, cls_R, cls_fpr, cls_f1, cls_acc*

In [ ]:
!python multitasks/val.py --img 512 --weights {weights_path} --data ../datasets/esmart_context/data.yaml  --batch-size 32 --only_cls_eval --temperature 3


### The detections on esmart_wip

*Class, Images, Instances, P, R, mAP50, mAP50-95*

In [ ]:
weights_path = 'runs/train-mlt/exp162/weights/best.pt'
!python multitasks/val.py --img 512 --weights {weights_path} --data ../datasets/esmart_wip/data.yaml --batch-size 32 --only_det_eval

### OR both tasks on esmart_hybrid

In [ ]:
!python multitasks/val.py --img 512 --weights {weights_path} --data ../datasets/hybrid/data.yaml --batch-size 32

-----------------

## Export the model (onnx supported)

In [ ]:
!python export.py --weights runs/train-mlt/exp205/weights/last.pt --include onnx